In [14]:
import os
import pandas as pd
import numpy as np
import networkx as nx
import stellargraph as sg
from stellargraph.layer.graphsage import MeanAggregator, AttentionalAggregator,\
                        MaxPoolingAggregator, MeanPoolingAggregator
from stellargraph import globalvar
from stellargraph.mapper import GraphSAGELinkGenerator
# from stellargraph.layer import GraphSAGE, link_classification
from datetime import datetime

In [2]:
from keras.models import load_model

In [5]:
DATA_DIR = os.getenv("DATA_DIR")
if DATA_DIR is None:
    DATA_DIR = '../../data'
PREDICT_DIR = os.path.join(DATA_DIR, "predict_network")
MODELS_DIR = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "models")

print(os.listdir(PREDICT_DIR))
print(os.listdir(MODELS_DIR))

predict_file = os.path.join(PREDICT_DIR, "predict_top50_vs_all.csv.gz")
node_data_file = os.path.join(PREDICT_DIR, "node_data_labelled_tfidf_2K.csv.gz")
model_file = os.path.join(MODELS_DIR, "graphsage_attentional_20meanpool_20e.h5")

['node_data_labelled_tfidf_2K.csv.gz', 'predict_top50_vs_all.csv.gz']
['.gitkeep', 'graphsage_20maxpool_20e_105_01d.h5', 'graphsage_attentional_20meanpool_20e.h5', 'graphsage_attentional_32layer.h5', 'vectorizer_v3.pickle', 'vectorizer.pickle', 'graphsage.h5']


In [18]:
predict_test = pd.read_csv(predict_file, compression="gzip")
node_data = pd.read_csv(node_data_file, compression="gzip", index_col=0)

In [19]:
node_data.head()

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
268029,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198002,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
157685,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
263649,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.68613,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166164,0.0,0.0,0.0,0.0,0.0,0.320345,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
model = load_model(model_file, custom_objects={'MeanPoolingAggregator': MeanPoolingAggregator})
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, 20, 2000)     0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           (None, 200, 2000)    0                                            
__________________________________________________________________________________________________
input_17 (InputLayer)           (None, 20, 2000)     0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 200, 2000)    0                                            
__________________________________________________________________________________________________
input_13 (

In [21]:
node_features = node_data[node_data.columns].values

In [73]:
batch_size = 1
num_samples = [20, 10]

def batched_predict(start,end):
    G = nx.from_pandas_edgelist(predict_test[['source','target','label']][start:end], edge_attr="label")

    for nid, f in zip(node_data.index, node_features):
        if nid in G.node.keys():
            G.node[nid][globalvar.TYPE_ATTR_NAME] = "page"  # specify node type
            G.node[nid]["feature"] = f

    G_predict = sg.StellarGraph(G, node_features="feature")

    edge_ids_test = [(e[0],e[1]) for e in G_predict.edges()]

    predict_gen = GraphSAGELinkGenerator(G_predict, batch_size, num_samples).flow(edge_ids_test)

    print(datetime.now().strftime("%H:%M:%S"))

    pred = model.predict_generator(predict_gen, verbose=1, workers=8, use_multiprocessing=True, 
                                   max_queue_size=100)

    print(datetime.now().strftime("%H:%M:%S"))
    print("Got maximum prediction: {}".format(max(pred)))
    return [p[0] for p in pred], edge_ids_test

In [74]:
def chunker(length, chunksize):
    return [[i,i+chunksize] if i+chunksize < length else [i,length-1] for i in range(0, length, chunksize)]

In [81]:
predictions = []
test_ids = []
for indices in chunker(predict_test[0:20000].shape[0], 10000):
    print(indices[0],indices[1])
    pred_batch, id_batch = batched_predict(indices[0],indices[1])
#     print(pred_batch)
    predictions.extend(pred_batch)
    test_ids.extend(id_batch)

0 10000
11:07:08
10000/10000 [==============================] - 268s 27ms/step
11:11:35
Got maximum prediction: [0.7309102]
10000 19999
11:11:38
9999/9999 [==============================] - 204s 20ms/step
11:15:02
Got maximum prediction: [0.7309175]


In [82]:
predictions[0:10]

[0.7272296,
 0.7272296,
 0.7272296,
 0.7272296,
 0.7272296,
 0.7272296,
 0.7272296,
 0.7272296,
 0.72920096,
 0.72920096]

In [83]:
tempo = predict_test[0:len(predictions)].copy(deep=True)
tempo['predictions'] = predictions 
tempo['test_ids'] = test_ids
print(tempo.shape)
tempo.sort_values("predictions",ascending=False)

(19999, 9)


,source_cid,target_cid,source_bp,target_bp,source,target,label,predictions,test_ids
14814,b220a437-0d51-4390-9993-63345d0c83ad,5d66c32e-7631-11e4-a3cb-005056011aef,/sign-in-universal-credit,/government/publications/crb-structure-and-sal...,136,151262,go_to,0.730918,"(136, 151262)"
14815,b220a437-0d51-4390-9993-63345d0c83ad,db00eb66-870f-44f2-b0a2-203b15e01dbd,/sign-in-universal-credit,/aaib-reports/nieuport-24-replica-n153js-11-ap...,136,151263,go_to,0.730918,"(136, 151263)"
14813,b220a437-0d51-4390-9993-63345d0c83ad,ff3758c9-cd5e-423e-b32f-6a3a2725865f,/sign-in-universal-credit,/employment-tribunal-decisions/mrs-k-armer-v-m...,136,151261,go_to,0.730918,"(136, 151261)"
14812,b220a437-0d51-4390-9993-63345d0c83ad,5eb61537-7631-11e4-a3cb-005056011aef,/sign-in-universal-credit,/government/collections/building-regulations-c...,136,4600,go_to,0.730918,"(136, 4600)"
14811,b220a437-0d51-4390-9993-63345d0c83ad,d94ac9e1-8a0b-4097-8076-5ef747964ed6,/sign-in-universal-credit,/government/news/191-million-cash-boost-for-th...,136,151260,go_to,0.730918,"(136, 151260)"
14810,b220a437-0d51-4390-9993-63345d0c83ad,2b55fad0-0cdc-4123-968b-e7516fe9dfb2,/sign-in-universal-credit,/government/publications/climate-change-levy-a...,136,151259,go_to,0.730918,"(136, 151259)"
14809,b220a437-0d51-4390-9993-63345d0c83ad,440308ac-527b-4c16-ae06-1ab06aa5c446,/sign-in-universal-credit,/employment-tribunal-decisions/ms-j-cole-v-gov...,136,151258,go_to,0.730918,"(136, 151258)"
14808,b220a437-0d51-4390-9993-63345d0c83ad,5ee3ff65-7631-11e4-a3cb-005056011aef,/sign-in-universal-credit,/government/publications/costs-and-benefits-to...,136,151257,go_to,0.730918,"(136, 151257)"
1591,b220a437-0d51-4390-9993-63345d0c83ad,b9e80176-0c92-4d67-a515-30cf8c4c3413,/sign-in-universal-credit,/government/consultations/uk-standards-for-mic...,136,139771,go_to,0.730910,"(136, 139771)"
1590,b220a437-0d51-4390-9993-63345d0c83ad,00a01551-b3c7-448d-9fe0-67d55dfaa51c,/sign-in-universal-credit,/government/news/welsh-secretary-demands-a-str...,136,139770,go_to,0.730910,"(136, 139770)"


In [77]:
predict_test['predictions'] = predictions 
predict_test['test_ids'] = test_ids
# print(tempo.shape)

ValueError: Length of values does not match length of index